In [5]:
from fsplit.filesplit import Filesplit
import pickle
import os

In [6]:
W_Data_Dict = {}
if(not os.path.isfile('Appdata/W_Data_Dict.pickle4')):
    fs = Filesplit()

    def merge_cb(f, s):
        print("file: {0}, size: {1}".format(f, s))
    fs.merge(input_dir="Appdata", callback=merge_cb)

else:
    print('Data File Available')

file: Appdata\W_Data_Dict.pickle, size: 800980971


In [7]:
open_file_ = open("Appdata/W_Data_Dict.pick4", "rb")
W_Data_Dict = pickle.load(open_file_)
open_file_.close()
print('File Loaded')

File Loaded


In [9]:
lons = []
lats = []
keys = list(W_Data_Dict)
for key in keys:
    temp = key.split(',')
    lons.append(float(temp[0]))
    lats.append(float(temp[1]))

print(len(lons))
print(len(lats))

lons = set(lons)
lats = set(lats)

print(len(lons))
print(len(lats))

15953
15953
471
239


#### Write and Read Location Index

In [21]:
import csv

rows = [lats,lons]
with open('meta.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(rows)

In [5]:
import csv

rows = []
with open('meta.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        rows.append(row)
lats=rows[0]
lons=rows[2]
lats =[float(i) for i in lats]
lons =[float(i)  for i in lons]


In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json

cred = credentials.Certificate('nodemcu-tester-firebase-adminsdk-p8xun-4c3dfad99c.json')
# firebase_admin.initialize_app(cred)
db = firestore.client()
for i,key in enumerate(W_Data_Dict):
    print(i)
    list_upload = {}
    for doy,data in enumerate(W_Data_Dict[key]):
        list_upload[str(doy+1)] = json.dumps(data)
    doc_ref = db.collection("H2O").document(key)
    doc_ref.set(list_upload)


In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json

def listToString(s): 
    str1 = " " 
    opt =["%.4f" % i for i in s]
    return (str1.join(opt))

cred = credentials.Certificate('nodemcu-tester-firebase-adminsdk-p8xun-4c3dfad99c.json')
#firebase_admin.initialize_app(cred)
db = firestore.client()
list_upload = {}

list_upload["LATS"] = listToString(lats)
list_upload["LONS"] = listToString(lons)
doc_ref = db.collection("H2O").document("DATA")
doc_ref.set(list_upload)

update_time {
  seconds: 1643394877
  nanos: 514835000
}

In [9]:
meta_data = db.collection("H2O").document("DATA").get()
meta_data = meta_data.to_dict()
lats=meta_data["LATS"].split(' ')
lons=meta_data["LONS"].split(' ')
lats_get =[float(i) for i in lats]
lons_get =[float(i)  for i in lons]

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json

cred = credentials.Certificate('nodemcu-tester-firebase-adminsdk-p8xun-4c3dfad99c.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
def getData(lat, lon, date):
    meta_data = db.collection("H2O").document("DATA").get()
    meta_data = meta_data.to_dict()
    lats=meta_data["LATS"].split(' ')
    lons=meta_data["LONS"].split(' ')
    lats_get =[float(i) for i in lats]
    lons_get =[float(i)  for i in lons]

    lats = np.sort(np.array(lats_get))
    lat_cat = np.searchsorted(lats, lat)
    lat_cat = round(lats[lat_cat], 4)

    lons = np.sort(np.array(lons_get))
    lon_cat = np.searchsorted(lons, lon)
    lon_cat = round(lons[lon_cat], 4)

    location = str('%.4f' % lon_cat)+','+str('%.4f' % lat_cat)
    doy = datetime.datetime.strptime(date, '%Y-%m-%d').timetuple().tm_yday
    # print(location)
    w_data_year = db.collection("H2O").document(location).get()
    w_data_year = w_data_year.to_dict()

    w_data_dict = {}
    header = ['YEAR', 'MO', 'DY', 'T2M', 'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE', 'T2M_MAX', 'T2M_MIN', 'QV2M', 'RH2M',
              'PRECTOTCORR', 'PS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN']
    for i in range(0, 7):
        w_data = json.loads(w_data_year[str(doy-6+i)])
        for ind, val in enumerate(header[3:], start=3):
            key = val+'_'+str(i)
            w_data_dict[key] = float(w_data[ind])
    entries_to_remove = ('T2M_RANGE_0', 'QV2M_0', 'QV2M_1', 'T2M_RANGE_2', 'QV2M_2',
                         'QV2M_3', 'T2M_RANGE_4', 'T2M_RANGE_5', 'PRECTOTCORR_5', 'T2M_RANGE_6')
    for k in entries_to_remove:
        w_data_dict.pop(k, None)
    final_list = [lon, lat, cal2jd(date)]+list(w_data_dict.values())
    # print(final_list)
    return final_list

In [10]:
w_data_year = db.collection("H2O").document("-109.8032,46.3254").get()
w_data_year = w_data_year.to_dict()
print(w_data_year)

None
